In [1]:
import torch
from torch import nn

In [2]:
class DoubleConv(nn.Module):
    
    def __init__(self, in_size: int, out_size: int):
        super(DoubleConv, self).__init__()

        self.conv_net = nn.Sequential(
            nn.Conv1d(in_size, out_size, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_size, out_size, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv_net(x)


In [3]:
class DownSample(nn.Module):

    def __init__(self, in_size: int, out_size: int):
        super(DownSample, self).__init__()

        self.conv = DoubleConv(in_size, out_size)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

    def forward(self, x):
        down = self.conv(x)
        p = self.pool(down)
        return down, p



In [4]:
class UpSample():

    def __init__(self, in_size: int, out_size: int):
        super(UpSample, self).__init__()

        self.up = nn.ConvTranspose1d(in_size, in_size//2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_size, out_size)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x1, x2])
        return self.conv(x)



In [5]:
class UNet(nn.Module):

    def __init__(self, in_size: int, num_classes: int):
        super(UNet, self).__init__()

        self.down_conv_1 = DownSample(in_size, 64)
        self.down_conv_2 = DownSample(64, 128)
        self.down_conv_3 = DownSample(128, 256)

        self.bottle_nack = DoubleConv(256, 512)

        self.up_conv_1 = UpSample(512, 256)
        self.up_conv_2 = UpSample(256, 128)
        self.up_conv_3 = UpSample(128, 64)

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes),
            nn.Softmax()
        )

    def forward(self, x):

        down_1, p1 = self.down_conv_1(x)
        down_2, p2 = self.down_conv_2(p1)
        down_3, p3 = self.down_conv_3(p2)

        b = self.bottle_nack(p3)

        up_1 = self.up_conv_1(b, down_3)
        up_2 = self.up_conv_2(up_1, down_2)
        up_3 = self.up_conv_3(up_2, down_1)

        out = self.fc(up_3)
        return out

In [6]:
model = UNet(32, 12)
print(model)


UNet(
  (down_conv_1): DownSample(
    (conv): DoubleConv(
      (conv_net): Sequential(
        (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU(inplace=True)
        (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (3): ReLU(inplace=True)
      )
    )
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (down_conv_2): DownSample(
    (conv): DoubleConv(
      (conv_net): Sequential(
        (0): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU(inplace=True)
        (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
        (3): ReLU(inplace=True)
      )
    )
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (down_conv_3): DownSample(
    (conv): DoubleConv(
      (conv_net): Sequential(
        (0): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU(inplace=True)
     

In [7]:
YW3e8Sq

NameError: name 'YW3e8Sq' is not defined